In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import re



In [2]:
def get_system_from_cif(cif_file):
    # expr =r"(?:-?\s*[\w\.\']+)(?:\s*\+\s*\d+\/\d+)?(?:,\s*-?\s*[\w\.\']+)(?:\s*\+\s*\d+\/\d+)?(?:,\s*-?\s*[\w\.\']+)(?:\s*\+\s*\d+\/\d+)?"
    # res = re.findall(expr, cif_file, re.DOTALL)
    # return res
    # Находим блок координат с помощью регулярного выражения
    # Находим блок координат с помощью регулярного выражения
    # Находим блок координат с помощью регулярного выражения
    block_pattern = re.compile(r'loop_\s*_symmetry_equiv_pos_site_id\s*_symmetry_equiv_pos_as_xyz\s*(.*?)\n(?=loop_|$)', re.DOTALL)
    block_match = block_pattern.search(cif_file)
    
    if block_match:
        # Извлекаем найденный блок
        block_content = block_match.group(1)
        
        # Разделяем блок на строки
        lines = block_content.strip().split('\n')
        
        # Используем регулярное выражение для извлечения только координат
        coordinates = []
        for line in lines:
            # Удаляем цифры в начале строки и одинарные кавычки
            cleaned_line = re.sub(r'\d+\s', '', line).replace("'", "")
            # Добавляем координаты в список
            if cleaned_line.strip():  # Пропускаем пустые строки
                coordinates.append(cleaned_line.strip())
        
        return coordinates
    else:
        return None

In [31]:

#Размножение координаты по ПСТ
def expand_coordinates(system_points, coord):
    x, y, z = coord
    new_coords = []
    for pos in system_points:
        new_pos = pos.replace('x', str(x)).replace('y', str(y)).replace('z', str(z))
        parts = new_pos.split(',')
        new_part = []
        for part in parts:
            # if '-' in part:
            #     sign = -1
            #     part = part.replace('-', '')
            else:
                sign = 1
            if '/' in part:
                if '+' in part:
                #     parts_plus = part.split('+')
                #     numerator = parts_plus[0].split('/')
                #     denominator = parts_plus[1].split('/')
                #     numerator_value = float(numerator[0]) / float(numerator[1]) if len(numerator) == 2 else 0
                #     denominator_value = float(denominator[0]) / float(denominator[1]) if len(denominator) == 2 else 0
                #     value = sign * (numerator_value + denominator_value)
                # else:
                #     fraction = part.split('/')
                #     value = sign * float(fraction[0]) / float(fraction[1])
                    sub_parts = part.split('+')
                    value = 0.5 + float(sub_parts[1])
                elif '-' in part:
                    sub_parts = part.split('-')
                    value = 0.5 - float(sub_parts[1])
            else:
                value = sign * float(part)
            new_part.append(value)
        new_coords.append(new_part)
    return new_coords

In [4]:
data = pd.read_csv('cubic_groups.csv')


conn = mysql.connector.connect(host='localhost', user='user', password='12345', database='NNCDB')
cursor = conn.cursor()
id = data['CCDC_ID'][154]
query = 'SELECT CIF_FILE FROM CIFS WHERE ID = %s'
cursor.execute(query, (int(id),))
resp_res = cursor.fetchall()
test_str = resp_res[0][0]
cursor.close()
conn.close()

In [18]:
print(get_system_from_cif(test_str))

['x,y,z', '-x,1/2-y,z', 'x,-y,1/2-z', '1/2-x,y,-z', 'z,x,y', 'y,z,x', '-z,1/2-x,y', '1/2-y,z,-x', 'z,-x,1/2-y', '-y,1/2-z,x', '1/2-z,x,-y', 'y,-z,1/2-x', '1/2+x,1/2+y,1/2+z', '1/2-x,-y,1/2+z', '1/2+x,1/2-y,-z', '-x,1/2+y,1/2-z', '1/2+z,1/2+x,1/2+y', '1/2+y,1/2+z,1/2+x', '1/2-z,-x,1/2+y', '-y,1/2+z,1/2-x', '1/2+z,1/2-x,-y', '1/2-y,-z,1/2+x', '-z,1/2+x,1/2-y', '1/2+y,1/2-z,-x']


In [33]:
sys = ['x,y,z', '-x,1/2-y,z', 'x,-y,1/2-z', '1/2-x,y,-z', 'z,x,y', 'y,z,x', '-z,1/2-x,y', '1/2-y,z,-x', 'z,-x,1/2-y', '-y,1/2-z,x', '1/2-z,x,-y', 'y,-z,1/2-x', '1/2+x,1/2+y,1/2+z', '1/2-x,-y,1/2+z', '1/2+x,1/2-y,-z', '-x,1/2+y,1/2-z', '1/2+z,1/2+x,1/2+y', '1/2+y,1/2+z,1/2+x', '1/2-z,-x,1/2+y', '-y,1/2+z,1/2-x', '1/2+z,1/2-x,-y', '1/2-y,-z,1/2+x', '-z,1/2+x,1/2-y', '1/2+y,1/2-z,-x']
coord = [1.0,1.0,1.0]
pst = expand_coordinates(sys, coord)
pst

ValueError: could not convert string to float: ''